<a href="https://colab.research.google.com/github/Sri0035/Smart-Parking-System/blob/main/SmartParkingCategorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas scikit-learn

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler

def preprocess_data(file_path):
    try:
        # Load the CSV data
        df = pd.read_csv(file_path)

        # Check if 'price' and 'number_plate' columns exist
        if 'price' not in df.columns or 'number_plate' not in df.columns:
            raise ValueError("The dataset must contain 'price' and 'number_plate' columns.")

        # Optional: Use additional features like 'year' if available
        features = ['price']
        if 'year' in df.columns:
            features.append('year')

        # Encode the 'number_plate' into numerical values
        label_encoder = LabelEncoder()
        df['number_plate_encoded'] = label_encoder.fit_transform(df['number_plate'])
        features.append('number_plate_encoded')

        # Define the feature set X and the target set y
        X = df[features]
        y = categorize_by_price_or_model(df)

        return X, y

    except FileNotFoundError:
        print(f"Error: File {file_path} not found.")
        return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

def categorize_by_price_or_model(df):
    # Categorize each car based on price
    conditions = [
        (df['price'] >= 50000),              # High End if price >= 50,000 USD
        (df['price'] >= 20000) & (df['price'] < 50000),  # Medium End if 20,000 <= price < 50,000
        (df['price'] < 20000)                # Low End if price < 20,000 USD
    ]
    categories = ['Low End', 'Medium End', 'High End']
    return pd.cut(df['price'], bins=[0, 20000, 50000, float('inf')], labels=categories)

def train_knn_model(X, y):
    try:
        # Split the data into training and testing sets (80% train, 20% test)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Standardize the features for better performance
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Train KNN classifier
        knn = KNeighborsClassifier(n_neighbors=5)  # K=5
        knn.fit(X_train_scaled, y_train)

        # Predict and evaluate the model
        y_pred = knn.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"Model Accuracy: {accuracy * 100:.2f}%")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        return knn, scaler

    except Exception as e:
        print(f"An error occurred during model training: {e}")
        return None, None

def categorize_new_data(file_path, knn_model, scaler):
    try:
        # Load the dataset for prediction
        X, _ = preprocess_data(file_path)

        # Standardize the features
        X_scaled = scaler.transform(X)

        # Predict categories using the trained model
        predictions = knn_model.predict(X_scaled)
        X['Predicted_Category'] = predictions

        # Display results
        print("\nCategorized Vehicles:")
        print(X[['price', 'number_plate_encoded', 'Predicted_Category']])

        # Save results to a new CSV file
        X.to_csv('categorized_vehicles_knn_with_number_plates.csv', index=False)
        print("\nResults saved to 'categorized_vehicles_knn_with_number_plates.csv'.")

    except Exception as e:
        print(f"An error occurred during prediction: {e}")

# Main program
if __name__ == "__main__":
    file_path = input("Enter the path to your vehicle dataset CSV file (e.g., 'vehicles.csv'): ")

    # Preprocess data
    X, y = preprocess_data(file_path)
    if X is not None and y is not None:
        # Train the KNN model
        knn_model, scaler = train_knn_model(X, y)

        # Predict categories for new data
        if knn_model is not None and scaler is not None:
            categorize_new_data(file_path, knn_model, scaler)



Enter the path to your vehicle dataset CSV file (e.g., 'vehicles.csv'): /content/synthetic_vehicle_data_with_number_plates (1).csv
Model Accuracy: 95.00%

Classification Report:
              precision    recall  f1-score   support

    High End       0.98      1.00      0.99        57
     Low End       0.91      0.97      0.94        72
  Medium End       0.97      0.89      0.93        71

    accuracy                           0.95       200
   macro avg       0.95      0.95      0.95       200
weighted avg       0.95      0.95      0.95       200


Categorized Vehicles:
     price  number_plate_encoded Predicted_Category
0     5409                   240            Low End
1    16087                   471            Low End
2    62280                   279           High End
3    16731                   817            Low End
4    18261                   212            Low End
..     ...                   ...                ...
995  43737                   487         Medium End
99